<a href="https://colab.research.google.com/github/dinaldoap/jit-sdp-nn/blob/master/notebook/mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import random
from scipy.stats import mstats

from jitsdp import metrics
from jitsdp.pipeline import Pipeline

In [23]:
df = pd.read_csv('https://raw.githubusercontent.com/dinaldoap/jit-sdp-data/master/jenkins.csv')
df.head()

,fix,ns,nd,nf,entropy,la,ld,lt,ndev,age,nuc,exp,rexp,sexp,author_date_unix_timestamp,classification,contains_bug
0,False,7.0,7.0,7.0,2.641604,9.0,9.0,426.428571,100.0,0.000093,1.0,5171.0,30.227271,1472.714286,1555326371,None,False
1,False,7.0,7.0,7.0,2.750000,8.0,8.0,426.428571,100.0,6.314775,2.0,5170.0,29.227271,1471.714286,1555326363,None,False
2,False,1.0,1.0,2.0,0.906580,15.0,44.0,96.000000,4.0,0.034722,2.0,629.0,14.828373,414.000000,1554971763,None,False
3,False,1.0,1.0,1.0,0.000000,0.0,0.0,40.000000,1.0,0.000012,1.0,4.0,3.058824,3.000000,1554969774,None,False
4,False,1.0,2.0,4.0,1.662506,14.0,10.0,67.000000,6.0,21.280683,4.0,3.0,2.058824,2.000000,1554967752,Feature Addition,False


In [24]:
#df = df.sample(frac=1)
label_col = 'contains_bug'
features_cols = ['fix', 'ns', 'nd', 'nf', 'entropy', 'la', 'ld', 'lt', 'ndev', 'age', 'nuc', 'exp', 'rexp', 'sexp', 'classification']
X = df[features_cols]
X['fix'] = X['fix'].astype('int')
df_classification = pd.get_dummies(X, columns=['classification'])
X = pd.concat([X, df_classification], axis='columns')
X = X.drop(['classification'], axis='columns')
X = X.values
y = df[label_col]
y = y.astype('category')
y = y.cat.codes
y = y.values

/home/pytorch/.conda/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [25]:
print(X[:1])
print(y[:1])

[[0.00000000e+00 7.00000000e+00 7.00000000e+00 7.00000000e+00
  2.64160417e+00 9.00000000e+00 9.00000000e+00 4.26428571e+02
  1.00000000e+02 9.25925926e-05 1.00000000e+00 5.17100000e+03
  3.02272705e+01 1.47271429e+03 0.00000000e+00 7.00000000e+00
  7.00000000e+00 7.00000000e+00 2.64160417e+00 9.00000000e+00
  9.00000000e+00 4.26428571e+02 1.00000000e+02 9.25925926e-05
  1.00000000e+00 5.17100000e+03 3.02272705e+01 1.47271429e+03
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
  0.00000000e+00 0.00000000e+00]]
[0]


In [26]:
test_index = int( len(X) * 0.9 )
X_train, y_train = X[:test_index], y[:test_index]
X_test, y_test = X[test_index:], y[test_index:]

In [27]:
print(X_train[:1])
print(X_test[:1])

[[0.00000000e+00 7.00000000e+00 7.00000000e+00 7.00000000e+00
  2.64160417e+00 9.00000000e+00 9.00000000e+00 4.26428571e+02
  1.00000000e+02 9.25925926e-05 1.00000000e+00 5.17100000e+03
  3.02272705e+01 1.47271429e+03 0.00000000e+00 7.00000000e+00
  7.00000000e+00 7.00000000e+00 2.64160417e+00 9.00000000e+00
  9.00000000e+00 4.26428571e+02 1.00000000e+02 9.25925926e-05
  1.00000000e+00 5.17100000e+03 3.02272705e+01 1.47271429e+03
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
  0.00000000e+00 0.00000000e+00]]
[[0.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  0.00000000e+00 1.00000000e+00 1.00000000e+00 2.31000000e+02
  2.00000000e+00 8.17129630e-03 1.00000000e+00 2.29000000e+03
  2.35138387e+02 1.96900000e+03 0.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 0.00000000e+00 1.00000000e+00
  1.00000000e+00 2.31000000e+02 2.00000000e+00 8.17129630e-03
  1.00000000e+00 2.29000000e+03 2.35138387e+02 1.96900000e+03
  0.00000000e+00 0.00000000e+00 0.00

In [28]:
mean_train = X_train.mean(axis=0)
std_train = X_train.std(axis=0)

In [29]:
print(mean_train)
print(std_train)

[2.84157779e-01 1.61084038e+00 2.58937109e+00 4.11079441e+00
 7.49641664e-01 4.94600037e+01 2.76527676e+01 7.55224305e+02
 2.09663019e+01 9.03703380e+01 1.96731335e+00 1.81907199e+03
 7.69168636e+01 9.19113438e+02 2.84157779e-01 1.61084038e+00
 2.58937109e+00 4.11079441e+00 7.49641664e-01 4.94600037e+01
 2.76527676e+01 7.55224305e+02 2.09663019e+01 9.03703380e+01
 1.96731335e+00 1.81907199e+03 7.69168636e+01 9.19113438e+02
 2.84157779e-01 1.50606841e-01 3.42497242e-02 4.78576683e-01
 1.47572637e-02 3.76517102e-02]
[4.51012345e-01 1.63796058e+00 6.67531787e+00 2.43475764e+01
 1.13389245e+00 5.71750268e+02 5.85715662e+02 1.10935403e+03
 2.76076700e+01 2.26346855e+02 7.26033575e+00 2.06378941e+03
 7.91356267e+01 1.36351174e+03 4.51012345e-01 1.63796058e+00
 6.67531787e+00 2.43475764e+01 1.13389245e+00 5.71750268e+02
 5.85715662e+02 1.10935403e+03 2.76076700e+01 2.26346855e+02
 7.26033575e+00 2.06378941e+03 7.91356267e+01 1.36351174e+03
 4.51012345e-01 3.57665235e-01 1.81869955e-01 4.99540

In [30]:
X_train = (X_train - mean_train) / std_train
X_test = (X_test - mean_train) / std_train

In [31]:
print(X_train[:1])
print(X_test[:1])

[[-0.63004435  3.29016442  0.66073691  0.11866502  1.66855552 -0.07076517
  -0.03184611 -0.29638485  2.86274423 -0.39925558 -0.13323259  1.62416184
  -0.58999461  0.40601106 -0.63004435  3.29016442  0.66073691  0.11866502
   1.66855552 -0.07076517 -0.03184611 -0.29638485  2.86274423 -0.39925558
  -0.13323259  1.62416184 -0.58999461  0.40601106 -0.63004435 -0.42108325
  -0.18831986  1.0438052  -0.12238588 -0.19779997]]
[[-0.63004435 -0.3729274  -0.23809669 -0.12776608 -0.66112237 -0.08475729
  -0.04550462 -0.47254915 -0.68699394 -0.39921989 -0.13323259  0.22818608
   1.99937159  0.76998718 -0.63004435 -0.3729274  -0.23809669 -0.12776608
  -0.66112237 -0.08475729 -0.04550462 -0.47254915 -0.68699394 -0.39921989
  -0.13323259  0.22818608  1.99937159  0.76998718 -0.63004435 -0.42108325
  -0.18831986  1.0438052  -0.12238588 -0.19779997]]


In [32]:
class Classifier(nn.Module):
  FILENAME = 'models/classifier.cpt'
  def __init__(self, input_size, hidden_size, drop_prob, epoch=None, val_gmean=None):
    super(Classifier, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.drop_prob = drop_prob
    self.epoch = epoch
    self.val_gmean = val_gmean
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.fcout = nn.Linear(hidden_size, 1)
    self.dropout = nn.Dropout(drop_prob)
  
  def forward(self, x):
    x = torch.relu(self.fc1(x))
    x = self.dropout(x)
    x = torch.sigmoid(self.fcout(x))
    return x

  def save(self):
    checkpoint = {
        'input_size': self.input_size,
        'hidden_size': self.hidden_size,
        'drop_prob': self.drop_prob,
        'val_gmean': self.val_gmean,
        'epoch': self.epoch,
        'state_dict': self.state_dict()
    }
    with open(Classifier.FILENAME, 'wb') as f:
      torch.save(checkpoint, f)

  def load(self):
    with open(Classifier.FILENAME, 'rb') as f:
      checkpoint = torch.load(f)
      self.input_size = checkpoint['input_size']
      self.hidden_size = checkpoint['hidden_size']
      self.drop_prob = checkpoint['drop_prob']
      self.epoch = checkpoint['epoch']
      self.val_gmean = checkpoint['val_gmean']
      self.load_state_dict(checkpoint['state_dict'])

In [33]:
classifier = Classifier(input_size=X.shape[1], hidden_size=X.shape[1], drop_prob=0.5)
classifier

Classifier(
  (fc1): Linear(in_features=34, out_features=34, bias=True)
  (fcout): Linear(in_features=34, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [34]:
classifier.save()
classifier.load()
classifier

Classifier(
  (fc1): Linear(in_features=34, out_features=34, bias=True)
  (fcout): Linear(in_features=34, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [35]:
criterion = nn.BCELoss()
optimizer = optim.Adam(params=classifier.parameters(), lr=0.003)

In [36]:
pipeline = Pipeline(steps=[], classifier=classifier, optimizer=optimizer, criterion=criterion, max_epochs=200, fading_factor=0.9999)
pipeline.train(X_train, y_train)

rain loss: 0.5531444301101011, Train g-mean: 0.7041673842410323, Val g-mean: 0.7354159034053153
Epoch: 16, Train loss: 0.5452996027517839, Train g-mean: 0.7097098312810494, Val g-mean: 0.7415447320878773
Epoch: 17, Train loss: 0.5496033655070048, Train g-mean: 0.7037022432688945, Val g-mean: 0.7496923972539953
Epoch: 18, Train loss: 0.5377251954834247, Train g-mean: 0.710941313505955, Val g-mean: 0.7473618514079278
Epoch: 19, Train loss: 0.5383792890237962, Train g-mean: 0.7089247713746331, Val g-mean: 0.7442710953382234
Epoch: 20, Train loss: 0.5397580970890684, Train g-mean: 0.7082231661548223, Val g-mean: 0.7451103279664498
Epoch: 21, Train loss: 0.5468890516920155, Train g-mean: 0.7086659802260893, Val g-mean: 0.7378379259401373
Epoch: 22, Train loss: 0.5410651545467623, Train g-mean: 0.7047279462929005, Val g-mean: 0.7522709292727274
Epoch: 23, Train loss: 0.5434670638853226, Train g-mean: 0.7086951652758462, Val g-mean: 0.7520782265370977
Epoch: 24, Train loss: 0.5363920167771514

In [37]:
  def evaluate(pipeline):
    train_gmean, train_recalls = pipeline.evaluate(X_train, y_train)
    test_gmean, test_recalls = pipeline.evaluate(X_test, y_test)
    print('Epoch: {}'.format(pipeline.epoch))
    print('Train g-mean: {}, recalls: {}'.format(train_gmean.item(), train_recalls))
    print('Test g-mean: {}, recalls: {}'.format(test_gmean.item(), test_recalls))

In [38]:
print('Last classifier')
evaluate(pipeline)

Last classifier
Epoch: 199
Train g-mean: 0.7461736734404283, recalls: [0.73941845 0.75299061]
Test g-mean: 0.7407253009874877, recalls: [0.78547772 0.69852264]


In [39]:
print('Best classifier')
pipeline.load()
evaluate(pipeline)

Best classifier
Epoch: 197
Train g-mean: 0.7466168316428287, recalls: [0.72771996 0.7660044 ]
Test g-mean: 0.7418633688268692, recalls: [0.77461644 0.7104952 ]
